In [2]:
# automatic reloading of modules when they change.
%load_ext autoreload
%autoreload 2

In [3]:
from qiskit_metal import draw, Dict
from qiskit_metal.qlibrary.core import BaseQubit
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
from qiskit_metal.draw import LineString
from qiskit_metal.qlibrary.core.qroute import QRouteLead, QRoutePoint, QRoute
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from  qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL
from qiskit_metal.qlibrary.user_components.my_qcomponent import TransmonCrossRound,TransmonCrossRound_v1
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from collections import  OrderedDict
import  math
import numpy as np
import qiskit_metal as metal

In [4]:
metal.about()


Qiskit Metal        0.1.2

Basic
____________________________________
 Python              3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
 Platform            Windows AMD64
 Installation path   C:\Users\CJZ\anaconda3\lib\site-packages\qiskit_metal

Packages
____________________________________
 Numpy               1.21.5
 Qutip               4.7.1

Rendering
____________________________________
 Matplotlib          3.5.2

GUI
____________________________________
 PySide2 version     5.15.2.1
 Qt version          5.15.2
 SIP version         4.19.13

IBM Quantum Team


'\nQiskit Metal        0.1.2\n\nBasic\n____________________________________\n Python              3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]\n Platform            Windows AMD64\n Installation path   C:\\Users\\CJZ\\anaconda3\\lib\\site-packages\\qiskit_metal\n\nPackages\n____________________________________\n Numpy               1.21.5\n Qutip               4.7.1\n\nRendering\n____________________________________\n Matplotlib          3.5.2\n\nGUI\n____________________________________\n PySide2 version     5.15.2.1\n Qt version          5.15.2\n SIP version         4.19.13\n\nIBM Quantum Team'

In [3]:
import qiskit_metal as metal
design = metal.designs.DesignPlanar()
gui = metal.MetalGUI(design)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'
design.chips['main']['material'] = 'sapphire'
design.variables.cpw_gap='4 um'
design.variables.cpw_width = '8 um'
design.overwrite_enabled = True

In [4]:
xmon_round = TransmonCrossRound_v1(design,'my_xmon_round', options=Dict(radius = '0 um', connection_pads=dict(
                      bus_02=dict(claw_length='61um',ground_spacing='3um', claw_width='8um',claw_gap='4um',connector_location='90'),
                      )))
# Add hanger consisting of capacitively coupled transmission lines.
TQ1 = CoupledLineTee(design, 'TQ1', options=Dict(pos_x='15.02um', pos_y='1.611 mm', coupling_length='284um', coupling_space='3um',
                                                 prime_width = '10um', prime_gap = '5um', second_width = '8um', second_gap = '4um', down_length='200um',fillet='34um',mirror=True,
                                                 open_termination = False))

pos_y = TQ1.options.pos_y

otg1 = OpenToGround(design, 'otg1', options=Dict(pos_x='-0.347mm', pos_y=pos_y,  orientation='180'))
otg2 = OpenToGround(design,
                    'otg2',
                    options=Dict(pos_x='0.347mm', pos_y=pos_y,width='10um',gap='5um',))


gui.rebuild()
gui.autoscale()

In [138]:
# TQ1.options

In [5]:
ops_oL = Dict(hfss_wire_bonds=True,
              pin_inputs=Dict(start_pin=Dict(component='TQ1',
                                             pin='prime_start'),
                              end_pin=Dict(component='otg1', pin='open')),trace_width='10um',trace_gap='5um')

ops_oR = Dict(hfss_wire_bonds=True,
              pin_inputs=Dict(start_pin=Dict(component='TQ1', pin='prime_end'),
                              end_pin=Dict(component='otg2', pin='open')),trace_width='10um',trace_gap='5um')

cpw_openLeft = RouteStraight(design, 'cpw_openLeft', options=ops_oL)
cpw_openRight = RouteStraight(design, 'cpw_openRight', options=ops_oR)

# -----------------------------------------------------------------------------------------------------
fillet = 0.034
ops = Dict(fillet=fillet)

jogsS = OrderedDict()
jogsS[0] = ["R", '0.1um']

qubit_cpw_length =4*design.components['my_xmon_round'].parse_options().connection_pads.bus_02.claw_width-0.004
TQ_length =design.components['TQ1'].parse_options().down_length+design.components['TQ1'].parse_options().coupling_length-2*fillet+(math.pi/2)*fillet
whole_length = 4.5975452-0.0028520*2-0.002633*2
total_length = whole_length-(qubit_cpw_length+TQ_length)

start_straight = 0.15-qubit_cpw_length+fillet

options = Dict(total_length=total_length,
                hfss_wire_bonds=True,
                pin_inputs=Dict(end_pin=Dict(component='TQ1',
                                               pin='second_end'),
                                start_pin=Dict(component='my_xmon_round', pin='bus_02')),
               meander=Dict(spacing='68.000001um'),
                lead=Dict(start_straight=start_straight,start_jogged_extension=jogsS),
                **ops)



meanderQ= RouteMeander(design, 'meanderQ1', options=options)
gui.rebuild()
gui.autoscale()



In [163]:
# design.components['my_xmon_round'].parse_options().connection_pads.bus_02.claw_width
# design.components['TQ1'].parse_options().coupling_length
# design.components['TQ1'].parse_options().down_length

0.3

In [201]:
cpw_openRight.options

{'chip': 'main',
 'layer': '1',
 'pin_inputs': {'start_pin': {'component': 'TQ1', 'pin': 'prime_end'},
  'end_pin': {'component': 'otg2', 'pin': 'open'}},
 'fillet': '0',
 'lead': {'start_straight': '0mm',
  'end_straight': '0mm',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '7mm',
 'trace_width': '10um',
 'hfss_wire_bonds': True,
 'q3d_wire_bonds': False,
 'trace_gap': '5um',
 '_actual_length': '0.04797999999999997 mm'}

In [9]:
a_gds = design.renderers.gds
a_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': 2.0,
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'negative_mask': {'main': [1]},
 'fabricate': 'True',
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': '../resources/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': '199',
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '100um',
  'view_in_file': {'main': {1: False}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '200um'},
 'no_cheese': {'datatype': '99',
  'buffer': '25um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: False}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [11]:
a_gds.options['short_segments_to_not_fillet'] = 'True'
a_gds.options['fabricate']='True'
scale_fillet = 2.0
a_gds.options['check_short_segments_by_scaling_fillet'] = scale_fillet
a_gds.options.negative_mask = {'main': [1,]}
# a_gds.options.cheese.shape = '1'
# a_gds.options.cheese.delta_x = '200um'
# a_gds.options.cheese.delta_y = '200um'
# a_gds.options.cheese.edge_nocheese = '200 um'
# a_gds.options.no_cheese.buffer = '50 um'
# a_gds.options.cheese.cheese_1_radius = '25 um'
a_gds.options.cheese.view_in_file =  {'main': {1: False, },}
a_gds.options.no_cheese.view_in_file =  {'main': {1: False}, }
a_gds.export_to_gds('100qubit_simulation_v7.gds')

05:42PM 06s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"C:\Users\CJZ\PycharmProjects\resources".


1